In [1]:
from tqdm import tqdm_notebook
import numpy as np, pandas as pd, os
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
#basic
import numpy as np, pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from tqdm import tqdm_notebook
from sklearn.decomposition import PCA
#Pretreatment
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.feature_selection import VarianceThreshold
#model
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn import svm, neighbors, linear_model, neural_network
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.naive_bayes import MultinomialNB, GaussianNB, BernoulliNB
from sklearn.ensemble import RandomForestClassifier
import warnings
warnings.filterwarnings('ignore')

In [3]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')
train.head()

,id,muggy-smalt-axolotl-pembus,dorky-peach-sheepdog-ordinal,slimy-seashell-cassowary-goose,snazzy-harlequin-chicken-distraction,frumpy-smalt-mau-ordinal,stealthy-beige-pinscher-golden,chummy-cream-tarantula-entropy,hazy-emerald-cuttlefish-unsorted,nerdy-indigo-wolfhound-sorted,...,wheezy-myrtle-mandrill-entropy,wiggy-lilac-lemming-sorted,gloppy-cerise-snail-contributor,woozy-silver-havanese-gaussian,jumpy-thistle-discus-sorted,muggy-turquoise-donkey-important,blurry-buff-hyena-entropy,bluesy-chocolate-kudu-fepid,gamy-white-monster-expert,target
0,707b395ecdcbb4dc2eabea00e4d1b179,-2.070654,1.018160,0.228643,0.857221,0.052271,0.230303,-6.385090,0.439369,-0.721946,...,0.351895,0.618824,-1.542423,0.598175,0.611757,0.678772,0.247059,-0.806677,-0.193649,0
1,5880c03c6582a7b42248668e56b4bdec,-0.491702,0.082645,-0.011193,1.071266,-0.346347,-0.082209,0.110579,-0.382374,-0.229620,...,-0.645115,-1.246090,2.613357,-0.479664,1.581289,0.931258,0.151937,-0.766595,0.474351,0
2,4ccbcb3d13e5072ff1d9c61afe2c4f77,-1.680473,0.860529,-1.076195,0.740124,3.678445,0.288558,0.515875,0.920590,-1.223277,...,0.516422,0.130521,-0.459210,2.028205,-0.093968,-0.218274,-0.163136,-0.870289,0.064038,1
3,e350f17a357f12a1941f0837afb7eb8d,0.183774,0.919134,-0.946958,0.918492,0.862278,1.155287,0.911106,0.562598,-1.349685,...,-1.168967,1.385089,-0.353028,3.316150,-0.524087,-0.794327,3.936365,0.682989,-2.521211,0
4,a8f910ea6075b6376af079055965ff68,-0.203933,-0.177252,0.368074,-0.701320,-1.104391,0.735760,0.894273,-1.375826,-5.144946,...,-0.279168,1.544356,2.959727,1.641201,-0.130818,-0.264292,-0.748668,0.964218,0.087079,0


In [4]:
cols = [c for c in train.columns if c not in ['id', 'target', 'wheezy-copper-turtle-magic']]

# 훈련 테스트 데이터 반올림
train1 = np.round(train,3)
test1 = np.round(test, 3)

In [5]:
test['target']=0
data = pd.concat([pd.DataFrame(train1), pd.DataFrame(test1)],axis=0)
features = data[cols].columns

#변수별 unique_value_counts
var_stats = {}
hist_df = pd.DataFrame()
for var in tqdm_notebook(features):
    var_stats = data[var].value_counts()
    hist_df[var] = pd.Series(data[var]).map(var_stats)
data2 = pd.concat([data,hist_df],axis=1)

In [10]:
data2.head()

,baggy-champagne-capuchin-discard,baggy-copper-oriole-dummy,baggy-coral-bandicoot-unsorted,baggy-mustard-collie-hint,beady-aquamarine-affenpinscher-grandmaster,beady-asparagus-opossum-expert,beady-champagne-bullfrog-grandmaster,beady-lilac-hornet-expert,beady-mauve-frog-distraction,beady-orange-binturong-golden,...,goopy-lavender-wolverine-fimbus,wheezy-myrtle-mandrill-entropy,wiggy-lilac-lemming-sorted,gloppy-cerise-snail-contributor,woozy-silver-havanese-gaussian,jumpy-thistle-discus-sorted,muggy-turquoise-donkey-important,blurry-buff-hyena-entropy,bluesy-chocolate-kudu-fepid,gamy-white-monster-expert
0,0.531,-0.228,-0.081,0.142,0.310,0.082,0.832,0.115,-1.830,-0.343,...,15,132,119,49,122,107,122,135,87,145
1,-1.841,0.910,-0.444,-0.534,-0.614,0.532,-3.065,0.217,0.982,-3.144,...,74,111,71,9,107,53,106,140,104,135
2,0.567,-0.981,1.515,-1.975,-5.847,0.428,0.388,-0.703,0.829,1.421,...,134,118,118,134,22,163,136,137,102,152
3,0.696,-0.412,-1.145,0.305,0.552,-0.454,-0.516,2.921,-2.232,-0.751,...,35,93,53,136,7,137,96,5,106,13
4,-2.107,3.185,0.552,-0.457,0.575,-1.184,1.609,-0.166,0.178,2.692,...,93,130,40,12,40,163,146,111,93,142


In [11]:
#훈련 테스트데이터 분리
train2 = data2[:train1.shape[0]]; test2 = data2[train1.shape[0]:]

In [12]:
#target변수 삭제
test2.drop('target',axis=1,inplace=True)

In [14]:
oof = np.zeros(len(train))
preds = np.zeros(len(test))

# BUILD 512 SEPARATE MODELS
for i in tqdm_notebook(range(512)):
    # ONLY TRAIN WITH DATA WHERE WHEEZY EQUALS I
    train10 = train[train['wheezy-copper-turtle-magic']==i]
    train3 = train2[train2['wheezy-copper-turtle-magic']==i]
    test3 = test2[test2['wheezy-copper-turtle-magic']==i]
    idx1 = train3.index; idx2 = test2.index
    train3.reset_index(drop=True,inplace=True)
    
    
    useful = train10[cols].columns[np.std(train10[cols])>1.5]
    train4 = train3[useful]
    test4 = test3[useful]
    # STRATIFIED K-FOLD
    skf = StratifiedKFold(n_splits=11, random_state=42, shuffle=True)
    for train_index, test_index in skf.split(train4, train3['target']):
        
        # MODEL AND PREDICT WITH QDA
        clf = LGBMClassifier()
        clf.fit(train4.loc[train_index,:],train3.loc[train_index]['target'])
        oof[idx1[test_index]] = clf.predict_proba(train4.loc[test_index,:])[:,1]

    
    #if i%64==0: print(i)
        
# PRINT CV AUC
auc = roc_auc_score(train['target'],oof)
print('LGBoost scores CV =',round(auc,5))

LGBoost scores CV = 0.81785


In [15]:
oof = np.zeros(len(train))
preds = np.zeros(len(test))

# BUILD 512 SEPARATE MODELS
for i in tqdm_notebook(range(512)):
    # ONLY TRAIN WITH DATA WHERE WHEEZY EQUALS I
    train10 = train[train['wheezy-copper-turtle-magic']==i]
    train3 = train2[train2['wheezy-copper-turtle-magic']==i]
    test3 = test2[test2['wheezy-copper-turtle-magic']==i]
    idx1 = train3.index; idx2 = test2.index
    train3.reset_index(drop=True,inplace=True)
    
    
    useful = train10[cols].columns[np.std(train10[cols])>1.5]
    train4 = train3[useful]
    test4 = test3[useful]
    # STRATIFIED K-FOLD
    skf = StratifiedKFold(n_splits=11, random_state=42, shuffle=True)
    for train_index, test_index in skf.split(train4, train3['target']):
        
        # MODEL AND PREDICT WITH QDA
        clf = QuadraticDiscriminantAnalysis(0.1)
        clf.fit(train4.loc[train_index,:],train3.loc[train_index]['target'])
        oof[idx1[test_index]] = clf.predict_proba(train4.loc[test_index,:])[:,1]

    
    #if i%64==0: print(i)
        
# PRINT CV AUC
auc = roc_auc_score(train['target'],oof)
print('QDA scores CV =',round(auc,5))

QDA scores CV = 0.92654
